# IMPORT LIBARY


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tkinter as tk

# LOAD DATASETS


In [ ]:
data_path = "datasets/supermarket_sales.csv"

df = pd.read_csv(data_path)


# melihat 5 data teratas
df.head(5)

In [ ]:
df.shape

In [ ]:
# melihat tipe data pada tiap kolom header
df.info()

In [ ]:
# mendeskirpsikan isi dari datasets
df.describe()

# DATA PREPROCESSING


In [ ]:
df.isna().sum()
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df = df.drop('gross margin percentage', axis=1)


In [ ]:
# mengecek struktur table setelah kolom gross margin precentage di drop
df.head(10)

In [ ]:
label, feature = df.shape
# melihat jumlah feature
print(feature)

# melihat jumlah label
print(label)

In [ ]:
# meliaht jumlah data unique
df.nunique()

In [ ]:
unique_counts = df.nunique()
unique_df = unique_counts.reset_index()
unique_df.columns = ["feature", "jumlah_unique"]

plt.figure(figsize=(10, 6))

sns.barplot(
    data=unique_df,
    x="feature",
    y="jumlah_unique"
)

plt.title("Jumlah Data Unik Pada Setiap Feature")
plt.xlabel("Feature")
plt.ylabel("Jumlah Unique Value")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


# EXPLORATORY DATA ANALYS


In [ ]:
num_cols = ['Unit price', 'Quantity', 'Total', 'Rating']

plt.figure(figsize=(12, 8))
for i, col in enumerate(num_cols, 1):
    plt.subplot(2, 2, i)
    sns.histplot(df[col], kde=True)
    plt.title(f'Distribusi {col}')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.barplot(
    x='City',
    y='Total',
    data=df,
    estimator=sum
)

plt.title("Total Penjualan per Kota")
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(
    y='Product line',
    x='Total',
    data=df,
    estimator=sum
)

plt.title("Total Penjualan per Product Line")
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(
    x='Payment',
    data=df
)

plt.title("Jumlah Transaksi per Metode Pembayaran")
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.scatterplot(
    x='Quantity',
    y='Total',
    data=df
)

plt.title("Hubungan Quantity terhadap Total")
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(
    x='Gender',
    y='Rating',
    data=df
)

plt.title("Distribusi Rating Berdasarkan Gender")
plt.show()


In [ ]:
plt.figure(figsize=(6, 4))
sns.barplot(
    x='Customer type',
    y='Total',
    data=df,
    estimator=sum
)

plt.title("Total Penjualan Berdasarkan Tipe Customer")
plt.show()


In [ ]:
# membuat diagram korelasi antar data dalamx dengan data dalam y menggunakan heatmap
x_vars = ['Unit price', 'Quantity']
y_vars = ['Total']

sns.pairplot(
    df,
    x_vars=x_vars,
    y_vars=y_vars,
    height=4,
    aspect=1,
    kind='scatter'
)

plt.suptitle("Korelasi data X terhadap data Y", y=1.02)
plt.show()

In [ ]:
df_numeric = df.select_dtypes(include=['int64', 'float64'])

plt.figure(figsize=(10, 8))
sns.heatmap(
    df_numeric.corr(),
    annot=True,
    cmap='coolwarm',
    fmt='.2f'
)

plt.title("Heatmap Korelasi Data Numerik")
plt.show()

# LINEAR REGRETION ALGORYTHM


# GUI 

In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
from tkinter import *

class GUI: 
    def __init__(self, root, tabCtrl):
        self.root = root
        self.root.geometry("750x450")
        self.tabCtrl = tabCtrl
        self.root.title("GUI LINEAR REGRESION")
        self.tabCtrl.pack(expand=True, fill='both')
        
        self.layouts()
        
    def layouts(self):
        tab1 = tk.Frame(self.tabCtrl, bg="#e7eaeb")
        self.tabCtrl.add(tab1, text="Tab 1")
        
        tab2 = tk.Frame(self.tabCtrl, bg='#e7eaeb')
        self.tabCtrl.add(tab2, text="Tab 2")        
        
        
        
        
                
    
            
if __name__ == "__main__":
    root = tk.Tk()
    tabCtrl = ttk.Notebook(root)
    app = GUI(root, tabCtrl)
    root.mainloop()
    
    
    